In [ ]:
#pip install reportlab
import json
import glob
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import itertools
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.metrics import confusion_matrix
import pandas as pd
import random

In [ ]:
with open('dataset.json', 'r') as file:
    data = json.load(file)

print(data)

In [ ]:
len(data)

In [ ]:
# look at json
data[0]['cuisines'][0]['label']

In [ ]:
# extract images of one restaurant
one_restaurant_info=data[0]['more_details']['full_images']
one_restaurant_images=[i['image_id'] for i in one_restaurant_info]
one_restaurant_images

In [ ]:
# do this for all restaurants

res_image_dic={}
cuisine_dic={}
for res in data:
    # use cuisine type as restaurant 
    res_id= res['identifier']
    one_restaurant_info=res['more_details']['full_images']
    one_restaurant_images=[i['image_id'] for i in one_restaurant_info]
    if len(one_restaurant_images)!=0:

    # select randomly 1 images per restaurant, otherwise too many images
        selected=one_restaurant_images[-1]
    res_image_dic[res_id]=selected
    cuisine_dic[res_id] = res['cuisines'][0]['label']#

In [ ]:
df_cuisine=pd.DataFrame.from_dict(cuisine_dic, orient='index').reset_index()
df_cuisine.columns=['res_id', 'cuisine']
df_cuisine.groupby('cuisine').count().sort_values(by='res_id',ascending=False).iloc[:22]
# get the country names from top 20

In [ ]:
top_22 = list(df_cuisine.groupby('cuisine').count().sort_values(by='res_id',ascending=False).iloc[:22].reset_index()['cuisine'])
top_22 # however, this will give us the top pictures in terms of both food and interior!
# change to top 22 because I want to include amazing indian food :)


In [ ]:
country=[
 'Japanese',
 'Italian',
 'French',
 'Mediterranean Cuisine',
 'Modern British',
 'Modern French',
 'Cantonese',
 'Thai',
 'Chinese', 
 'Classic French', 'Indian']

In [ ]:
extract_relevant_id = list(df_cuisine[df_cuisine['cuisine'].isin(country)]['res_id'])

In [ ]:
res_image_dic=dict((k, res_image_dic[k]) for k in extract_relevant_id if k in res_image_dic)

In [ ]:
# list of all restaurant images
nested_list=list(res_image_dic.values())
image_names = nested_list.copy()
#image_names

In [ ]:
len(image_names) # okey number of images

In [ ]:
# test reading an image
folder_path = 'images'
image_path=folder_path + '/' + image_names[0] + '.jpg'
img=Image.open(image_path)
img

In [ ]:
# reading all images in a restaurant setting  (for relevant cuisines)
all_images={}
folder_path = 'images'
target_size = (256, 256) # need to resize because some images do not have the same shap

for name in image_names:
    file_name=name + '.jpg'
    image_path = folder_path + '/' + file_name
    try:
        with Image.open(image_path) as img:
            img = img.resize(target_size)
            image_array = np.array(img)

            all_images[name]=image_array
            #print(name)
    except:
        print(f"Error opening file '{file_name}', skipping...")
        continue

In [ ]:
test_image_list=[]

test_image_names=[]
for i in image_names: 
    #plt.imshow(all_images[i], cmap='gray')
    #plt.show()
    if all_images[i].shape == (256, 256, 3):
        test_image_list.append(all_images[i])
        test_image_names.append(i)
    #print(all_images[i].shape) 

In [ ]:
test=np.array(test_image_list)
test.shape

# Filter only food images using our model

In [ ]:
import keras

In [ ]:
food_interior_classifier = keras.models.load_model("food_interior_classifier.h5")
food_interior_classifier

In [ ]:
# Predict new images with the model
model_predictions=food_interior_classifier.predict(test)
# set the threshold
threshold = 0.3 # adjust this threshold to make sure what we predict as food is really food
# convert predicted probabilities to labels 0 and 1
predicted_labels = (model_predictions > threshold).astype(int)
# print the predicted labels

In [ ]:
# need to find a better threshold, many misclassifications:
# https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/

In [ ]:
# adddress class imbalance, we might want to query more images from less prevalent cuisines

In [ ]:
label_dict={0: 'food', 1: 'exterior'}
predicted=[]

food_only=[]
for ind, item in enumerate(test): # show images in order to check prediction 
    label=predicted_labels[ind][0]
    predicted.append(label)
    if label==0:
        food_only.append(item)
    #print('image ' + str(ind) + ' label:' + (label_dict[label]))
    #plt.imshow(item, cmap='gray')
    #plt.show()

In [ ]:
plt.imshow(food_only[-1], cmap='gray')
plt.show()

In [ ]:
round(1- sum(predicted)/len(predicted),2)
# percent predicted as food images

In [ ]:
def save_image(filename):
    p = PdfPages(filename)
    for ind, item in enumerate(food_only): # show images in order to check prediction 
        label=predicted_labels[ind][0]
        fig= plt.figure()
        plt.imshow(item, cmap='gray')
        plt.title('image ' + str(ind) + ' label:' + (label_dict[label]))
        fig.savefig(p, format='pdf') 
    # close the object
    p.close()  

In [ ]:
# filename = "cuisine_food_predicted.pdf"  
# save_image(filename) 